In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xarray as xr
import pingouin as pg
import os
from tqdm import tqdm
from os.path import join as pjoin
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr, spearmanr, zscore, wilcoxon
import sys
import pymannkendall as mk

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [4]:
sys.path.append('..')
import circletrack_neural as ctn
import circletrack_behavior as ctb
import pca_ica as ica
import plotting_functions as pf

In [5]:
## Create a list of sessions for easy plotting later and for changing column order
session_list = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1']
## Set path variables
dpath = '/media/caishuman/csstorage/phild/git/MazeProjects/output/assemblies'
behav_path = '/media/caishuman/csstorage/phild/git/MazeProjects/output/behav'
figure_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/python_analysis/progress_figures/cohort0'
## Create young and old mouse list
young_mice = ['Fornax', 'Janus', 'Lyra', 'Miranda', 'Naiad', 'Sao', 'Titania']
old_mice = ['Gemini', 'Oberon', 'Puck', 'Umbriel', 'Virgo', 'Ymir', 'Atlas']
## Create list of female or male young mice
young_female_mice = ['Fornax', 'Janus', 'Lyra', 'Sao', 'Titania']
male_young_mice = ['Miranda', 'Naiad']
x_bin_size = 1

The code block below bins the data by trials and determines ensemble trends.

In [54]:
## Set mouse list, path, and z_thresh for binarization
across_time = False
alpha_old = 0.05
analysis_type = 'max'
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_trial_times = {}
## Loop through each mouse
for mouse in tqdm(young_mice):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    trial_times = {}
    ## Load assemblies
    for session in session_list:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
            aligned_behavior = pd.read_feather(behav_file)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
            ## Get length of time for each trial
            time_diff = []
            for trial in np.unique(trials):
                ## Subset aligned_behavior by a given trial
                behavior = aligned_behavior.loc[trials == trial]
                ## Get the first and last timestamp to determine the window
                first_timestamp, last_timestamp = behavior.t.to_numpy()[0], behavior.t.to_numpy()[-1]
                ## Convert from ms to s
                first_timestamp = first_timestamp / 1000
                last_timestamp = last_timestamp / 1000
                ## Append to time_diff list
                time_diff.append(last_timestamp - first_timestamp)
            trial_times[session] = time_diff
        ## This is where the data gets binned either by even time intervals or by trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = analysis_type, 
                                                                                           zscored = True, alpha_old = alpha_old)  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', 
                                                                                               z_threshold = None, alpha_old = alpha_old, analysis_type = analysis_type)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_trial_times[mouse] = trial_times

 14%|█▍        | 1/7 [00:01<00:11,  1.96s/it]/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
 29%|██▊       | 2/7 [00:03<00:10,  2.01s/it]/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
 43%|████▎     | 3/7 [00:06<00:08,  2.08s/it]/home

In [55]:
aligned_behavior

,x,y,frame,distance,water,lick_port,lin_position,trials,t,animal,session,cohort
0,49.549973,1.182982,0,0.000000,False,-1,0.241292,0,0,Titania,Reversal1,cohort0
1,50.231045,1.330465,1,0.696858,False,-1,0.261010,0,56,Titania,Reversal1,cohort0
2,51.144539,1.551503,2,0.939856,False,-1,0.287581,0,120,Titania,Reversal1,cohort0
3,52.329394,1.878092,3,1.229040,False,-1,0.322291,0,184,Titania,Reversal1,cohort0
4,54.579260,2.648011,4,2.377956,False,-1,0.389378,0,248,Titania,Reversal1,cohort0
...,...,...,...,...,...,...,...,...,...,...,...,...
26325,7.387647,22.151012,28941,0.024131,False,-1,5.088365,26,1800011,Titania,Reversal1,cohort0
26326,7.417603,22.127245,28942,0.038239,False,-1,5.089278,26,1800074,Titania,Reversal1,cohort0
26327,7.397499,22.139105,28943,0.023341,False,-1,5.088770,26,1800138,Titania,Reversal1,cohort0
26328,7.310417,22.260471,28945,0.078668,False,-1,5.084778,26,1800266,Titania,Reversal1,cohort0


In [56]:
behav_file = pjoin(behav_path, '{}_{}.feat'.format('Janus', 'Reversal1'))
aligned_behavior = pd.read_feather(behav_file)

In [57]:
aligned_behavior.loc[aligned_behavior['water'] == True]

,x,y,frame,distance,water,lick_port,lin_position,trials,t,animal,session,cohort
0,60.965287,8.035072,1,0.199575,True,-1,0.645913,0,19,Janus,Reversal1,cohort0
88,43.432794,70.255749,89,0.198795,True,3,3.072077,0,5889,Janus,Reversal1,cohort0
547,60.869650,7.984402,552,0.249186,True,0,0.642697,1,36755,Janus,Reversal1,cohort0
728,42.788138,70.454946,734,0.182908,True,-1,3.090370,1,48883,Janus,Reversal1,cohort0
2459,60.011240,7.370138,2478,0.084850,True,-1,0.610934,3,165154,Janus,Reversal1,cohort0
...,...,...,...,...,...,...,...,...,...,...,...,...
25322,43.653893,70.417520,25511,0.651905,True,3,3.066251,53,1700690,Janus,Reversal1,cohort0
25750,60.840779,7.864508,25942,0.450642,True,0,0.639847,54,1729428,Janus,Reversal1,cohort0
26120,43.995155,70.262872,26315,0.988093,True,-1,3.056421,54,1754291,Janus,Reversal1,cohort0
26505,61.187921,8.083515,26702,0.036262,True,-1,0.652118,55,1780082,Janus,Reversal1,cohort0


In [ ]:
## Plot peak activation of a fading ensemble
activations = mouse_binned_activations['Lyra']['Reversal1']
ensemble_number = 5
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = None,
                                  title = 'Fading Ensemble', y_title = 'Max Z Score', x_title = 'Trials', 
                                  file_name = 'max_strength_fading_forward_trials.png'.format(x_bin_size), marker_color = 'red')

In [ ]:
## Plot peak activation of an increasing ensemble
activations = mouse_binned_activations['Lyra']['Reversal1']
ensemble_number = 57
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = None,
                                  title = 'Increasing Ensemble', y_title = 'Max Z Score', x_title = 'Trials', 
                                  file_name = 'max_strength_increasing_forward_trials.png'.format(x_bin_size), marker_color = 'turquoise')

In [ ]:
## Plot peak activation of a no-trend ensemble
activations = mouse_binned_activations['Lyra']['Reversal1']
ensemble_number = 24
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = None,
                                  title = 'No Trend Ensemble', y_title = 'Max Z Score', x_title = 'Trials', 
                                  file_name = 'max_strength_notrend_forward_trials.png'.format(x_bin_size), marker_color = 'darkgrey')

In [284]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
# prop_df = prop_df.drop(labels = 6, axis = 0)
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines', line_color = 'darkgrey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Forward Trials',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
fig.write_image(pjoin(figure_path, 'proportion_fading_trials_mean.png'))

In [285]:
wilcoxon_result = wilcoxon(x = prop_df.Training4, y = prop_df.Reversal1, method = 'approx')
wilcoxon_result

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/scipy/stats/_morestats.py:3269: UserWarning:

Sample size too small for normal approximation.



WilcoxonResult(statistic=3.0, pvalue=0.06297905121445523)

In [ ]:
## Plot fading ensembles across all sessions
## Create empty dictionary
trends_dict = {}
## Loop through mouse_trends and extract the proportion of fading ensembles for each session (Training1, Training2, etc)
for key in mouse_trends:
    fading_ensembles = []
    for session in mouse_trends[key]:
        fading_ensembles.append(mouse_trends[key][session]['prop_decreasing'])
    trends_dict[key] = fading_ensembles
## Plot values for each mouse
fig = go.Figure()
x_axis = session_list
for key in trends_dict:
    fig.add_trace(go.Scatter(x = x_axis, y = trends_dict[key], name = key))
fig.update_layout(template = 'simple_white', height = 500, width = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensemble Proportion Across Sessions',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.show()
# fig.write_image('progress_figures/cohort0/by_trials_all.png')

The code below determines ensemble trends by binning by even time intervals.

In [2]:
## Set mouse list, path, and z_thresh for binarization
## Set x_bin_size for binning by time (in seconds)
x_bin_size = 5
across_time = True
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
## Loop through each mouse
for mouse in tqdm(young_mice):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    ## Load assemblies
    for session in session_list:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
            aligned_behavior = pd.read_feather(behav_file)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = 'max', zscored = True, alpha = 'sidak')  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', z_threshold = None)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends

NameError: name 'tqdm' is not defined

In [ ]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines', line_color = 'lightgrey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by {}s Bins'.format(x_bin_size),
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
# fig.write_image('progress_figures/cohort0/by_{}s_bins.png'.format(x_bin_size))

In [ ]:
wilcoxon_result = wilcoxon(x = prop_df.Training4, y = prop_df.Reversal1, method = 'approx')
wilcoxon_result

In [ ]:
## Plot peak activation of a fading ensemble
activations = mouse_binned_activations['Lyra']['Reversal1']
ensemble_number = 24
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = x_bin_size,
                                  title = 'Fading Ensemble', y_title = 'Max Z Score', x_title = 'Time (s)', file_name = 'max_strength_fading_{}bins.png'.format(x_bin_size))

In [ ]:
## Plot peak activation of an increasing ensemble
activations = mouse_binned_activations['Lyra']['Reversal1']
ensemble_number = 57
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = x_bin_size,
                                  title = 'Increasing Ensemble', y_title = 'Max Z Score', x_title = 'Time (s)', 
                                  file_name = 'max_strength_increasing_{}bins.png'.format(x_bin_size), marker_color = 'turquoise')

In [ ]:
## Plot activation strength of a no-trend ensemble
activations = mouse_binned_activations['Lyra']['Reversal1']
ensemble_number = 5
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = x_bin_size,
                                  title = 'No Trend Ensemble', y_title = 'Max Z Score', x_title = 'Time (s)', 
                                  file_name = 'max_strength_notrend_{}bins.png'.format(x_bin_size), marker_color = 'darkgrey')

In [ ]:
## Plot fading ensembles across all sessions
## Create empty dictionary
trends_dict = {}
## Loop through mouse_trends and extract the proportion of fading ensembles for each session (Training1, Training2, etc)
for key in mouse_trends:
    fading_ensembles = []
    for session in mouse_trends[key]:
        fading_ensembles.append(mouse_trends[key][session]['prop_decreasing'])
    trends_dict[key] = fading_ensembles
## Plot values for each mouse
fig = go.Figure()
x_axis = session_list
for key in trends_dict:
    fig.add_trace(go.Scatter(x = x_axis, y = trends_dict[key], name = key))
fig.update_layout(template = 'simple_white', height = 500, width = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensemble Proportion Across Sessions',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.show()
fig.write_image('progress_figures/cohort0/by_{}s_bins_all.png'.format(x_bin_size))

Analyze average trial times.

In [66]:
## Create a plot of average trial time across each session for all mice
df = pd.DataFrame(mouse_trial_times).T
mouse_avg_trial_times = {}
for mouse in df.index:
    session_dict = {}
    for session in df.columns:
        trial_times = df.loc[mouse, session]
        ## Bin trial time to see if trials are longer earlier in the session
        bin_size = 2
        if type(trial_times) is not list:
            pass
        else:
            ## Create bins
            bins = np.arange(1, len(trial_times), bin_size)
            ## Bin data
            binned = np.split(trial_times, bins)
            ## Take the average of each bin
            avg = [np.mean(bin) for bin in binned]
            session_dict[session] = avg
    mouse_avg_trial_times[mouse] = session_dict
## Avg time across each bin
session_averages = ctb.trial_averages(mouse_avg_trial_times, session_list)
avg_times = pd.DataFrame(mouse_avg_trial_times).T
avg_times = avg_times[session_list]
## Make figure
fig = make_subplots(rows = 2, cols = 3, subplot_titles = session_list, shared_yaxes = 'all')
for mouse in avg_times.index:
    for i, session in enumerate(avg_times.columns):
        data = avg_times.loc[mouse, session]
        x_axis = np.arange(0, len(data)*bin_size, bin_size)
        if i <= 2:
            fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.6, showlegend = False), row = 1, col = i+1)
        elif (i > 2) & (i <= 5):
            fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.6, showlegend = False), row = 2, col = i-2)
for i, session in enumerate(session_list):
    data = session_averages[session]
    x_axis = np.arange(0, len(data)*bin_size, bin_size)
    if i <= 2:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', opacity = 0.7, showlegend = False), row = 1, col = i+1)
    else:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', opacity = 0.7, showlegend = False), row = 2, col = i-2)
fig.update_layout(template = 'simple_white', width = 600, height = 600)
for col in np.arange(1, 4):
    fig.update_xaxes(title = 'Trial', row = 3, col = col)
    fig.update_yaxes(title = 'Trial Time (s)', row = col, col = 1)
fig.update_layout(
    title={
        'text': 'Binned Trial Time Across Session',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
for col in np.arange(1, 4):
    fig.update_xaxes(title = 'Trial', row = 2, col = col)
    fig.update_yaxes(title = 'Trial Time (s)', row = col, col = 1)
fig.show()
fig.write_image(pjoin(figure_path, 'trial_time_all.png'))

Create a scatterplot of TRIAL LENGTH vs max or mean activation strength. Takes the average of the max or mean values for a given trial time (since there are multiple values within a block of time).

In [192]:
mouse_results = {}
for mouse in young_mice:
    session_dictionary = {}
    for session in session_list:
        behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
        aligned_behavior = pd.read_feather(behav_file)
        ## Load assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        act = assemblies['activations'].values
        results = {'trial': [], 'trial_length': [], 'max': [], 'mean': [], 'median': []}
        for trial in np.unique(aligned_behavior.trials):
            behavior = aligned_behavior[aligned_behavior.trials == trial]
            first, last = (behavior.t.to_numpy()[0]/1000), (behavior.t.to_numpy()[-1]/1000) ## convert to seconds
            trial_length = last - first
            ## Get max and mean of activation strength within a trial
            trial_activation = act[:, aligned_behavior.trials == trial]
            results['max'].append(np.mean(np.nanmax(trial_activation, axis = 1)))
            results['mean'].append(np.mean(np.nanmean(trial_activation, axis = 1)))
            results['median'].append(np.mean(np.nanmedian(trial_activation, axis = 1)))
            ## Append trial number and trial length
            results['trial'].append(trial)
            results['trial_length'].append(trial_length)
        session_dictionary[session] = results
    mouse_results[mouse] = session_dictionary   
## Create dataframe
trial_df = pd.DataFrame()
for mouse in young_mice:
    for session in session_list:
        df = pd.DataFrame(mouse_results[mouse][session])
        df.insert(5, 'mouse', mouse)
        df.insert(6, 'session', session)
        trial_df = pd.concat([trial_df, df], ignore_index = True)

In [193]:
## Linear regression
lm_max = pg.linear_regression(trial_df['trial_length'], trial_df['max'], as_dataframe = False)
lm_mean = pg.linear_regression(trial_df['trial_length'], trial_df['mean'], as_dataframe = False)
lm_median = pg.linear_regression(trial_df['trial_length'], trial_df['median'], as_dataframe = False)
## Plot max
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Max Activation Strength')
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = trial_df['max'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = lm_max['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 400, y = 50, text = 'R<sup>2</sup> = {}'.format(round(lm_max['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 397, y = 20, text = 'p = 1.974e-244', showarrow = False)
fig.show()
# fig.write_image(pjoin(figure_path, 'trial_length_vs_max_allsessions.png'))

In [194]:
## Subset trial_df by trial times less than a minute
trial_len = 60
trial_df_sub = trial_df[trial_df.trial_length < trial_len]
lm_mean_sub = pg.linear_regression(trial_df_sub['trial_length'], trial_df_sub['max'], as_dataframe = False)
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Mean Activation Strength')
fig.add_trace(go.Scatter(x = trial_df_sub['trial_length'], y = trial_df_sub['max'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df_sub.trial, 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df_sub['trial_length'], y = lm_mean_sub['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.show()

In [39]:
## Linear regression
lm_trial = pg.linear_regression(trial_df.trial, trial_df.trial_length, as_dataframe = False)
## Plot max
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial', y_title = 'Trial Length')
fig.add_trace(go.Scatter(x = trial_df.trial, y = trial_df.trial_length, mode = 'markers', opacity = 0.7, showlegend = False, 
                         marker={'color': trial_df['max'], 'cmin': 0, 'cmax': 300, 'showscale': True, 'line': {'color': 'black', 'width' : 0.5}}))
# fig.add_trace(go.Scatter(x = trial_df.trial, y = lm_trial['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.show()

In [237]:
## Mean activation strength vs trial length
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Mean Activation Strength')
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = trial_df['mean'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = lm_mean['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 350, y = 12, text = 'R<sup>2</sup> = {}'.format(round(lm_mean['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 347, y = 11, text = 'p = 3.86e-07', showarrow = False)
fig.show()
fig.write_image(pjoin(figure_path, 'mean_activation_strength_allsessions.png'))

In [234]:
lm_mean

{'names': ['Intercept', 'trial_length'],
 'coef': array([2.8685075, 0.0061637]),
 'se': array([0.06326069, 0.00120925]),
 'T': array([45.34423043,  5.09713115]),
 'pval': array([3.79878275e-288, 3.86411507e-007]),
 'r2': 0.016197666970201863,
 'adj_r2': 0.015574218089954961,
 'CI[2.5%]': array([2.74442365, 0.0037918 ]),
 'CI[97.5%]': array([2.99259136, 0.0085356 ]),
 'df_model': 1,
 'df_resid': 1578,
 'residuals': array([ 6.38582749,  2.84952922, -0.49687392, ..., -0.05054507,
        -0.08284798, -0.67523771]),
 'X': array([[ 1.   , 66.128],
        [ 1.   , 89.968],
        [ 1.   , 29.871],
        ...,
        [ 1.   , 63.376],
        [ 1.   , 78.992],
        [ 1.   , 32.335]]),
 'y': array([9.66192822, 6.27257257, 2.55574949, ..., 3.20859315, 3.27254259,
        2.39257306]),
 'pred': array([3.27610072, 3.42304336, 3.05262342, ..., 3.25913822, 3.35539057,
        3.06781077])}

In [195]:
## Median activation strength vs trial length
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Median Activation Strength')
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = trial_df['median'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df['trial'], 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = lm_median['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 350, y = 0.40, text = 'R<sup>2</sup> = {}'.format(round(lm_median['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 350, y = 0.37, text = 'p = 0.380', showarrow = False)
fig.show()
# fig.write_image(pjoin(figure_path, 'trial_length_vs_median_allsessions.png'))

In [196]:
## Linear regression for JUST REVERSAL SESSION
reversal_df = trial_df[trial_df.session == 'Reversal1']
lm_max_reversal = pg.linear_regression(reversal_df.trial_length, reversal_df['max'], as_dataframe = False)
lm_mean_reversal = pg.linear_regression(reversal_df.trial_length, reversal_df['mean'], as_dataframe = False)
fig = pf.custom_graph_template('Reversal1', x_title = 'Trial Length (s)', y_title = 'Max Activation Strength')
fig.add_trace(go.Scatter(x = reversal_df.trial_length, y = reversal_df['max'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = reversal_df.trial_length, y = lm_max_reversal['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 150, y = 50, text = 'R<sup>2</sup> = {}'.format(round(lm_max_reversal['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 147, y = 20, text = 'p = 3.795e-102', showarrow = False)
fig.show()
# fig.write_image(pjoin(figure_path, 'trial_length_vs_max_reversal1.png'))

Plot for each animal trial vs trial length for two sessions (Training4 and Reversal1). Use Mann-Kendall test to see if the two are significantly correlated.

In [197]:
## Set which two sessions to compare
sessions = ['Training4', 'Reversal1']
## Subset
subset_df = trial_df[(trial_df['session'] == sessions[0]) | (trial_df['session'] == sessions[1])]
mouse_colors = ['grey', 'turquoise', 'violet', 'green', 'blue', 'black', 'red']
## Get mean values
first_session = subset_df[['trial', 'trial_length']][subset_df['session'] == sessions[0]].pivot(columns = 'trial')
avg_first = np.nanmean(first_session, axis = 0)
second_session = subset_df[['trial', 'trial_length']][subset_df['session'] == sessions[1]].pivot(columns = 'trial')
avg_second = np.nanmean(second_session, axis = 0)
## Create figure
fig = make_subplots(rows = 1, cols = 2, shared_yaxes = 'all', subplot_titles = sessions)
for i, mouse in enumerate(young_mice):
    fig.add_trace(go.Scatter(x = subset_df['trial'][(subset_df['session'] == sessions[0]) & (subset_df['mouse'] == mouse)], 
                             y = subset_df['trial_length'][(subset_df['session'] == sessions[0]) & (subset_df['mouse'] == mouse)], 
                             mode = 'lines', name = mouse, line_color = mouse_colors[i], opacity = 0.4), row = 1, col = 1)
trial_vector = np.arange(0, len(avg_first))
fig.add_trace(go.Scatter(x = trial_vector, y = avg_first, mode = 'markers', line_color = 'black', marker_size = 5, showlegend = False), row = 1, col = 1)
for i, mouse in enumerate(young_mice):
    fig.add_trace(go.Scatter(x = subset_df['trial'][(subset_df['session'] == sessions[1]) & (subset_df['mouse'] == mouse)], 
                            y = subset_df['trial_length'][(subset_df['session'] == sessions[1]) & (subset_df['mouse'] == mouse)], 
                            mode = 'lines', name = mouse, line_color = mouse_colors[i], opacity = 0.4, showlegend = False), row = 1, col = 2)
trial_vector = np.arange(0, len(avg_second))
fig.add_trace(go.Scatter(x = trial_vector, y = avg_second, mode = 'markers', line_color = 'black', marker_size = 5, showlegend = False), row = 1, col = 2)
fig.update_layout(template = 'simple_white')
fig.update_yaxes(title = 'Trial Length (s)')
fig.update_xaxes(title = 'Trial')
fig.show()
fig.write_image(pjoin(figure_path, 'T4andR1_trials_vs_trial_length.png'))

In [232]:
## Use Mann-Kendall test to ascertain trends
mouse_trends_kendall = {}
for mouse in young_mice:
    session_trends = {}
    for session_id in sessions:
        data = subset_df['trial_length'][(subset_df['mouse'] == mouse) & (subset_df['session'] == session_id)]
        session_trends[session_id] = mk.original_test(data, alpha = 0.05) 
    mouse_trends_kendall[mouse] = session_trends
## Collect tau values from Mann-Kendall test
taus = {'mouse': [], 'session': [], 'score': []}
for mouse in mouse_trends_kendall:
    for session in mouse_trends_kendall[mouse]:
        taus['mouse'].append(mouse)
        taus['session'].append(session)
        taus['score'].append(mouse_trends_kendall[mouse][session].s)
tau_df = pd.DataFrame(taus)
## Get prop_decreasing values
prop = {'mouse': [], 'session': [], 'prop_decreasing': []}
for mouse in mouse_trends:
    for session in mouse_trends[mouse]:
        prop['mouse'].append(mouse)
        prop['session'].append(session)
        prop['prop_decreasing'].append(mouse_trends[mouse][session]['prop_decreasing'])
prop_df_new = pd.DataFrame(prop)
prop_df_new = prop_df_new[(prop_df_new['session'] == 'Training4') | (prop_df_new['session'] == 'Reversal1')].reset_index()
## Combine tau_df and prop_decreasing column
trend_results = pd.concat([tau_df, prop_df_new['prop_decreasing']], axis = 1)

In [233]:
## Plot results
session_one_lm = pg.linear_regression(trend_results['score'][trend_results['session'] == sessions[0]], 
                                    trend_results['prop_decreasing'][trend_results['session'] == sessions[0]], as_dataframe = False)
session_two_lm = pg.linear_regression(trend_results['score'][trend_results['session'] == sessions[1]], 
                                    trend_results['prop_decreasing'][trend_results['session'] == sessions[1]], as_dataframe = False)
fig = make_subplots(rows = 1, cols = 2, shared_yaxes = 'all', subplot_titles = sessions)
## First session plot
fig.add_trace(go.Scatter(x = trend_results['score'][trend_results['session'] == sessions[0]], y = trend_results['prop_decreasing'][trend_results['session'] == sessions[0]],
                         mode = 'markers', showlegend = False), row = 1, col = 1)
fig.add_trace(go.Scatter(x = trend_results['score'][trend_results['session'] == sessions[0]], y = session_one_lm['pred'], mode = 'lines', line_color = 'red', showlegend = False))
## Second session plot
fig.add_trace(go.Scatter(x = trend_results['score'][trend_results['session'] == sessions[1]], y = trend_results['prop_decreasing'][trend_results['session'] == sessions[1]],
                         mode = 'markers', showlegend = False), row = 1, col = 2)
fig.add_trace(go.Scatter(x = trend_results['score'][trend_results['session'] == sessions[1]], y = session_two_lm['pred'], 
                         mode = 'lines', line_color = 'red', showlegend = False), row = 1, col = 2)
fig.update_layout(template = 'simple_white')
fig.update_yaxes(title = 'Proportion Fading')
fig.update_xaxes(title = "Mann-Kendal's Score")
fig.show()

In [222]:
## Combine tau_df and prop_decreasing column
trend_results = pd.concat([tau_df, prop_df_new['prop_decreasing']], axis = 1)
trend_results

,mouse,session,tau,prop_decreasing
0,Fornax,Training4,0.247480,0.000000
1,Fornax,Reversal1,0.103070,0.062500
2,Janus,Training4,-0.088710,0.000000
3,Janus,Reversal1,-0.362338,0.160000
4,Lyra,Training4,-0.561224,0.134328
5,Lyra,Reversal1,-0.437675,0.451613
6,Miranda,Training4,-0.399779,0.109375
7,Miranda,Reversal1,-0.389831,0.294118
8,Naiad,Training4,-0.203390,0.076923
9,Naiad,Reversal1,-0.320802,0.098361


The code below is used to plot each individual ensembles max activation strength within a given trial length. Less informative than the plots above.

In [ ]:
mouse_results = {}
for mouse in young_mice:
    session_dictionary = {}
    for session in session_list:
        behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
        aligned_behavior = pd.read_feather(behav_file)
        ## Load assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        act = assemblies['activations'].values
        results = {'trial': [], 'trial_length': [], 'max': [], 'mean': []}
        for trial in np.unique(aligned_behavior.trials):
            behavior = aligned_behavior[aligned_behavior.trials == trial]
            first, last = (behavior.t.to_numpy()[0]/1000), (behavior.t.to_numpy()[-1]/1000) ## convert to seconds
            trial_length = last - first
            ## Get max and mean of activation strength within a trial
            trial_activation = act[:, aligned_behavior.trials == trial]
            results['max'].append(np.nanmax(trial_activation, axis = 1))
            results['mean'].append(np.mean(trial_activation, axis = 1))
            ## Append trial number and trial length
            results['trial'].append(trial)
            results['trial_length'].append(trial_length)
        session_dictionary[session] = results
    mouse_results[mouse] = session_dictionary   
## Create dataframe
trial_df = pd.DataFrame()
for mouse in young_mice:
    for session in session_list:
        df = pd.DataFrame(mouse_results[mouse][session])
        df.insert(4, 'mouse', mouse)
        df.insert(5, 'session', session)
        df_melt = df['max'].apply(pd.Series) \
            .merge(df, right_index = True, left_index = True) \
            .drop(['max', 'mean'], axis = 1) \
            .melt(id_vars = ['trial', 'trial_length', 'mouse', 'session'], value_name = 'max')
        trial_df = pd.concat([trial_df, df_melt], ignore_index = True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = trial_df.trial_length, y = trial_df['max'], mode = 'markers'))
fig.update_layout(template = 'simple_white')
fig.show()

OLD MOUSE ANALYSES ARE BELOW:

Calculate the proportion of fading ensembles when binning by trials for old mice.

In [256]:
## Set mouse list, path, and z_thresh for binarization
across_time = False
alpha_old = 0.05
analysis_type = 'max'
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_trial_times = {}
## Loop through each mouse
for mouse in tqdm(old_mice):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    trial_times = {}
    ## Load assemblies
    for session in session_list:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
            aligned_behavior = pd.read_feather(behav_file)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
            ## Get length of time for each trial
            time_diff = []
            for trial in np.unique(trials):
                ## Subset aligned_behavior by a given trial
                behavior = aligned_behavior.loc[trials == trial]
                ## Get the first and last timestamp to determine the window
                first_timestamp, last_timestamp = behavior.t.to_numpy()[0], behavior.t.to_numpy()[-1]
                ## Convert from ms to s
                first_timestamp = first_timestamp / 1000
                last_timestamp = last_timestamp / 1000
                ## Append to time_diff list
                time_diff.append(last_timestamp - first_timestamp)
            trial_times[session] = time_diff
        ## This is where the data gets binned either by even time intervals or by trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = analysis_type, 
                                                                                           zscored = True, alpha_old = alpha_old)  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', 
                                                                                               z_threshold = None, alpha_old = alpha_old, analysis_type = analysis_type)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_trial_times[mouse] = trial_times

  0%|          | 0/7 [00:00<?, ?it/s]/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

 29%|██▊       | 2/7 [00:02<00:06,  1.33s/it]/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning:

divide by zero encountered in log1p

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarni

In [257]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
# prop_df = prop_df.drop(labels = 6, axis = 0)
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines', line_color = 'darkgrey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Forward Trials',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
# fig.write_image(pjoin(figure_path, 'proportion_fading_trials_mean_OLD.png'))

In [246]:
wilcoxon_result = wilcoxon(x = prop_df.Training4, y = prop_df.Reversal1, method = 'approx')
wilcoxon_result

WilcoxonResult(statistic=6.0, pvalue=0.6858304344516057)

Use the code below to create scatterplots of max, mean, median activation strength vs trial length.

In [161]:
mouse_results = {}
for mouse in old_mice:
    session_dictionary = {}
    for session in session_list:
        behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
        aligned_behavior = pd.read_feather(behav_file)
        ## Load assemblies
        assemblies = ica.load_session_assemblies(mouse, dpath, session)
        act = assemblies['activations'].values
        results = {'trial': [], 'trial_length': [], 'max': [], 'mean': [], 'median': []}
        for trial in np.unique(aligned_behavior.trials):
            behavior = aligned_behavior[aligned_behavior.trials == trial]
            first, last = (behavior.t.to_numpy()[0]/1000), (behavior.t.to_numpy()[-1]/1000) ## convert to seconds
            trial_length = last - first
            ## Get max and mean of activation strength within a trial
            trial_activation = act[:, aligned_behavior.trials == trial]
            results['max'].append(np.mean(np.nanmax(trial_activation, axis = 1)))
            results['mean'].append(np.mean(np.nanmean(trial_activation, axis = 1)))
            results['median'].append(np.mean(np.nanmedian(trial_activation, axis = 1)))
            ## Append trial number and trial length
            results['trial'].append(trial)
            results['trial_length'].append(trial_length)
        session_dictionary[session] = results
    mouse_results[mouse] = session_dictionary   
## Create dataframe
trial_df = pd.DataFrame()
for mouse in old_mice:
    for session in session_list:
        df = pd.DataFrame(mouse_results[mouse][session])
        df.insert(5, 'mouse', mouse)
        df.insert(6, 'session', session)
        trial_df = pd.concat([trial_df, df], ignore_index = True)

In [166]:
## Linear regression
lm_max = pg.linear_regression(trial_df['trial_length'], trial_df['max'], as_dataframe = False)
lm_mean = pg.linear_regression(trial_df['trial_length'], trial_df['mean'], as_dataframe = False)
lm_median = pg.linear_regression(trial_df['trial_length'], trial_df['median'], as_dataframe = False)
## Plot max
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Max Activation Strength')
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = trial_df['max'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = lm_max['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 350, y = 65, text = 'R<sup>2</sup> = {}'.format(round(lm_max['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 347, y = 20, text = 'p = 5.269e-145', showarrow = False)
fig.show()
fig.write_image(pjoin(figure_path, 'trial_length_vs_max_allsessions_OLD.png'))

In [252]:
## Plot mean
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Mean Activation Strength')
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = trial_df['mean'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df.trial, 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = lm_mean['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 350, y = 14, text = 'R<sup>2</sup> = {}'.format(round(lm_mean['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 347, y = 13, text = 'p = 3.86e-07', showarrow = False)
fig.show()
fig.write_image(pjoin(figure_path, 'trial_length_vs_mean_allsessions_OLD.png'))

In [169]:
## Median activation strength vs trial length
fig = pf.custom_graph_template('All Sessions', x_title = 'Trial Length (s)', y_title = 'Median Activation Strength')
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = trial_df['median'], mode = 'markers', opacity = 0.7, showlegend = False,
                         marker={'color': trial_df['trial'], 'colorscale': 'Blues', 'showscale': True, 'colorbar': {'title': 'Trial Number'}, 'line': {'color': 'black', 'width' : 0.5}}))
fig.add_trace(go.Scatter(x = trial_df['trial_length'], y = lm_median['pred'], mode = 'lines', line_color = 'red', showlegend = False))
fig.add_annotation(x = 300, y = 1.5, text = 'R<sup>2</sup> = {}'.format(round(lm_median['adj_r2'], 3)), showarrow = False)
fig.add_annotation(x = 297, y = 1.4, text = 'p = 9.471e-03', showarrow = False)
fig.show()
fig.write_image(pjoin(figure_path, 'trial_length_vs_median_allsessions_OLD.png'))

In [171]:
## Set which two sessions to compare
sessions = ['Training4', 'Reversal1']
## Subset
subset_df = trial_df[(trial_df['session'] == sessions[0]) | (trial_df['session'] == sessions[1])]
mouse_colors = ['grey', 'turquoise', 'violet', 'green', 'blue', 'black', 'red']
## Get mean values
first_session = subset_df[['trial', 'trial_length']][subset_df['session'] == sessions[0]].pivot(columns = 'trial')
avg_first = np.nanmean(first_session, axis = 0)
second_session = subset_df[['trial', 'trial_length']][subset_df['session'] == sessions[1]].pivot(columns = 'trial')
avg_second = np.nanmean(second_session, axis = 0)
## Create figure
fig = make_subplots(rows = 1, cols = 2, shared_yaxes = 'all', subplot_titles = sessions)
for i, mouse in enumerate(old_mice):
    fig.add_trace(go.Scatter(x = subset_df['trial'][(subset_df['session'] == sessions[0]) & (subset_df['mouse'] == mouse)], 
                             y = subset_df['trial_length'][(subset_df['session'] == sessions[0]) & (subset_df['mouse'] == mouse)], 
                             mode = 'lines', name = mouse, line_color = mouse_colors[i], opacity = 0.4), row = 1, col = 1)
trial_vector = np.arange(0, len(avg_first))
fig.add_trace(go.Scatter(x = trial_vector, y = avg_first, mode = 'markers', line_color = 'black', marker_size = 5, showlegend = False), row = 1, col = 1)
for i, mouse in enumerate(old_mice):
    fig.add_trace(go.Scatter(x = subset_df['trial'][(subset_df['session'] == sessions[1]) & (subset_df['mouse'] == mouse)], 
                            y = subset_df['trial_length'][(subset_df['session'] == sessions[1]) & (subset_df['mouse'] == mouse)], 
                            mode = 'lines', name = mouse, line_color = mouse_colors[i], opacity = 0.4, showlegend = False), row = 1, col = 2)
trial_vector = np.arange(0, len(avg_second))
fig.add_trace(go.Scatter(x = trial_vector, y = avg_second, mode = 'markers', line_color = 'black', marker_size = 5, showlegend = False), row = 1, col = 2)
fig.update_layout(template = 'simple_white')
fig.update_yaxes(title = 'Trial Length (s)')
fig.update_xaxes(title = 'Trial')
fig.show()
fig.write_image(pjoin(figure_path, 'T4andR1_trials_vs_trial_length_OLD.png'))

In [176]:
## Use Mann-Kendall test to ascertain trends
mouse_trends = {}
for mouse in old_mice:
    session_trends = {}
    for session_id in sessions:
        data = subset_df['trial_length'][(subset_df['mouse'] == mouse) & (subset_df['session'] == session_id)]
        session_trends[session_id] = mk.original_test(data, alpha = 0.05) 
    mouse_trends[mouse] = session_trends

All trial times for each session.

In [253]:
## Create a plot of average trial time across each session for all mice
df = pd.DataFrame(mouse_trial_times).T
mouse_avg_trial_times = {}
for mouse in df.index:
    session_dict = {}
    for session in df.columns:
        trial_times = df.loc[mouse, session]
        ## Bin trial time to see if trials are longer earlier in the session
        bin_size = 2
        if type(trial_times) is not list:
            pass
        else:
            ## Create bins
            bins = np.arange(1, len(trial_times), bin_size)
            ## Bin data
            binned = np.split(trial_times, bins)
            ## Take the average of each bin
            avg = [np.mean(bin) for bin in binned]
            session_dict[session] = avg
    mouse_avg_trial_times[mouse] = session_dict
## Avg time across each bin
session_averages = ctb.trial_averages(mouse_avg_trial_times, session_list)
avg_times = pd.DataFrame(mouse_avg_trial_times).T
avg_times = avg_times[session_list]
## Make figure
fig = make_subplots(rows = 2, cols = 3, subplot_titles = session_list, shared_yaxes = 'all')
# for mouse in avg_times.index:
#     for i, session in enumerate(avg_times.columns):
#         data = avg_times.loc[mouse, session]
#         x_axis = np.arange(0, len(data)*bin_size, bin_size)
#         if i <= 2:
#             fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.6, showlegend = False), row = 1, col = i+1)
#         elif (i > 2) & (i <= 5):
#             fig.add_trace(go.Scatter(x = x_axis, y = data, line_color = 'lightgrey', mode = 'lines', opacity = 0.6, showlegend = False), row = 2, col = i-2)
for i, session in enumerate(session_list):
    data = session_averages[session]
    x_axis = np.arange(0, len(data)*bin_size, bin_size)
    if i <= 2:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', opacity = 0.7, showlegend = False), row = 1, col = i+1)
    else:
        fig.add_trace(go.Scatter(x = x_axis, y = data, mode = 'lines', line_color = 'turquoise', opacity = 0.7, showlegend = False), row = 2, col = i-2)
fig.update_layout(template = 'simple_white', width = 600, height = 600)
for col in np.arange(1, 4):
    fig.update_xaxes(title = 'Trial', row = 3, col = col)
    fig.update_yaxes(title = 'Trial Time (s)', row = col, col = 1)
fig.update_layout(
    title={
        'text': 'Binned Trial Time Across Session',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
for col in np.arange(1, 4):
    fig.update_xaxes(title = 'Trial', row = 2, col = col)
    fig.update_yaxes(title = 'Trial Time (s)', row = col, col = 1)
fig.show()
# fig.write_image(pjoin(figure_path, 'trial_time_all_OLD.png'))

The code block below is used to replicate the PCA/ICA results from one of Will's mice.

In [ ]:
mouse = 'Fornax'
session = 'Reversal1'
minian_path = pjoin(dpath, 'processed/{}_{}.nc'.format(mouse, session))
minian_data = xr.open_dataset(minian_path)
## Detect assemblies
neural_data = minian_data.S_bin.values
smoothed_data = ctn.moving_average(neural_data, ksize = 5)
assemblies = ica.find_assemblies(smoothed_data, nullhyp = 'circ', n_shuffles = 500)
act = assemblies['activations']
## Load will's PCA/ICA result for Atlas Reversal1
dpath = pjoin(dpath, 'assemblies')
assemblies_will = ica.load_session_assemblies(mouse, dpath, session)
## Load behavior data
behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
aligned_behavior = pd.read_feather(behav_file)
## Get which timestamps are part of which trial for the aligned behavior data
trials = aligned_behavior.trials
## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
determined_trends = {}
trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', z_threshold = None)
## Create dictionary for Reversal1 trends
determined_trends[session] = trends
## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
proportion_dict = ica.calculate_proportions_ensembles(determined_trends)